In [1]:
#@title
! pip install plotly==4.14.3

     |████████████████████████████████| 13.2MB 218kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

# **Introduction**


The aim of this project is to develop a classification model for the discrimination between gamma-rays and hadron-rays. The dataset used is taken from the UCI database <a href="https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope">MAGIC Gamma Telescope</a> and it concerns data specifically created to simulate the presence of high energy gamma particles.  Gamma-rays are photons like the light rays but more "energetic". To observe them there are two ways: one is to use satellites in space and catch them before they pass through the earth atmosphere; the other way is to use earth detectors, like MAGIC, which observe the so-called *Cherenkov light*, that is the glow produced by paricles created when photons interact with the earth atmosphere.
<p align="center"><img src="https://magic.mpp.mpg.de/uploads/pics/MAGICTEST_2.jpg" width=700 height=200/></p>

The MAGIC (**M**ajor **A**tmospheric **G**amma-ray **I**maging **C**herenkov) Telescope is one of the major telescopes for observing gamma rays currently operative. It is made of two Imaging Atmospheric Cherenkov telescopes and it aims at detecting and study primary photons.<br/>
MAGIC is able to identigy the Cherenkov light thanks to the hundreds of mirrors it is made of, and therefore is able to capture it in pictures with a camera of less than nanosecond temporal resolution. 




<a href="https://home.infn.it/it/approfondimenti/esperimenti/1389-magic-major-atmospheric-gamma-imaging-cherenkov-telescope">Source INFN<a/>

In [1]:
#@title
import pandas as pd 
import numpy as np 
import csv 
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler, OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold,ParameterGrid, learning_curve,ShuffleSplit
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from imblearn.under_sampling import RandomUnderSampler
from statistics import mode
from collections import Counter
from pandas_profiling import ProfileReport
from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())
#pd.options.plotting.backend = "plotly"
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'cufflinks'

In [3]:
#@title
labels =  ["fLength","fWidth","fSize","fConc","fConc1", "fAsym","fM3Long","fM3Trans","fAlpha","fDist","class"]
df = pd.read_csv("magic04.data",names=labels, delimiter=",")
df

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


# **Data Exploration**

The data was generated to simulate registration of high energy gamma particles in an atmospheric Cherenkov telescope by a Monte Carlo program, Corsika. (<a href="https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope">UCI Dataset</a>).<br/>
Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background).

<div style="align:centre">
<img src="https://www.researchgate.net/profile/Mose-Mariotti/publication/3140370/figure/fig3/AS:667647221981189@1536190910397/Typical-shower-images-in-the-MAGIC-Telescope-corresponding-to-different-candidate.ppm" width=300 height=250/>
Tipical shower image taken from MAGIC <div/> 
<br/>

<img src="https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-030-24194-0_3/MediaObjects/483406_1_En_3_Fig6_HTML.png" width=300 height=250/><br/>
After some pre-processing these images can become ellipses, and its characteristics (also called Hillas parameters)can be used to performed discrimination. These parameters and other characteristcs are indeed the attributes of our dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19020 entries, 0 to 19019
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fLength   19020 non-null  float64
 1   fWidth    19020 non-null  float64
 2   fSize     19020 non-null  float64
 3   fConc     19020 non-null  float64
 4   fConc1    19020 non-null  float64
 5   fAsym     19020 non-null  float64
 6   fM3Long   19020 non-null  float64
 7   fM3Trans  19020 non-null  float64
 8   fAlpha    19020 non-null  float64
 9   fDist     19020 non-null  float64
 10  class     19020 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.6+ MB


As we can see, the dataset consists of 11 columns, considering also the ‘class’ column, which represents the target of the classification task. In particular, they represent:
1.   **fLength**: major axis of ellipse [mm]
2.   **fWidth**: minor axis of ellipse [mm]
3.   **fSize**: 10-log of sum of content of all pixels [in #phot]
4.   **fConc**: ratio of sum of two highest pixels over fSize [ratio]
5.  **fConc1**: ratio of highest pixel over fSize [ratio]
6.  **fAsym**: distance from highest pixel to center, projected onto major axis [mm]
7.  **fM3Long**: 3rd root of third moment along major axis [mm]
8.  **fM3Trans**: 3rd root of third moment along minor axis [mm]
9.  **fAlpha**: angle of major axis with vector to origin [deg]
10. **fDist**: distance from origin to center of ellipse [mm]
11. **class**: g,h -> gamma (signal), hadron (background)

The dataset have in total 19020 istances, with continous attributes that contains only numeric features, specifically floats. There is *not any missing value*. 

In [ ]:
print(f"Number of missing values: {df.isnull().values.sum()}")

Number of missing values: 0


As already quoted, the aim is distinguish between a gamma ray (class 'g') and a hadron (class 'h'). In particular 


*   **Gamma ray** is a penetreting form of electromagnetic radion araising from the radioactive decay of atomic nuclei.
*   **Hadron** is a subatomic particle made of two or more quarks held together by the strong force. The protons and the netrons are two types of hadron 


As we can see below, the features have all quite different statistics. This is a factor that makes me understand I have to normalize to improve performances with the models. 

In [ ]:
df.describe()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


Even though this is a dataset specifically created for simulation, it shows an imbalance between the two classes, beacuse it is technically easier to recreate gamma-ray characteristcs rathen than hadrons. But when dealing with real data it would actually be the opposite since hadrons represents the majority of the events.

In [ ]:
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}
px.histogram(df,x='class',color='class', width=400, height=300,color_discrete_map=color_discrete_map)

Just some featuers show a clear skewness. Skew indicates the distortion of a normal distribution, and it is quite common when dealing with real data, such as natural physics events.

In [ ]:
#@title
df_skew_kuto = pd.DataFrame()
df_skew_kuto['Skewness'] = df.skew()
df_skew_kuto['Kurtosis'] = df.kurtosis()
df_skew_kuto

,Skewness,Kurtosis
fLength,2.013652,4.970441
fWidth,3.371628,16.765407
fSize,0.875507,0.727278
fConc,0.485888,-0.521297
fConc1,0.685695,0.029391
fAsym,-1.046441,8.155330
fM3Long,-1.123078,4.670974
fM3Trans,0.120121,8.580352
fAlpha,0.850890,-0.533704
fDist,0.229587,-0.112577


Pairplots plots are a useful tool to visulize relationship between variables easily and graphically. The function provived by seaborn creates a grid axes such that every variable will be shared accross a sigle row in the x-axes and a single column in the y-axes. While the diagonal plots represent the univariate distribution of each variable.

In [ ]:
sns.set_context("paper", font_scale=0.9)
sns.pairplot(df, hue='class', plot_kws=dict(alpha=.50,s=70,linewidth=.5, edgecolor='w'),diag_kws=dict(shade=True),palette=["#588157","#f8961e"])

Output hidden; open in https://colab.research.google.com to view.

As we can see, most of the pairs of variables do not seem to have a particular correlation, but it pops up the positive correlation bewteen fConc and fConc1, and the negative correlatioin between fsize and fConc. 
Regarding the univariate distribution on the diagonal, we can see that, except for fAlpha, the distribution of class 'g' ad class 'h' have approximately similar shapes.

# **Data Preprocessing**

In machine learning algorithm the data pre-processing step is fundamental to create a reliable model. In this case, the dataset is already pretty clean, but there are still some adjustments to be done. The steps performed before starting the classification task are:
1. correlation among features, to see if there are some redundant variables
2. standardization, to make all variables comparable
3. outlier detection, to remove outliers which can make the classification unreliable
4. oversampling / undersampling, to "fix" the imbalance between the two classes in the original dataset
5. dimensionality reduction

The very first pre-processing step done is the are encoding  of the class labels using 0 and 1. Precesily:
- Class 'g' -> 0
- Class 'h' -> 1 

In [4]:
LE = LabelEncoder()
Y = LE.fit_transform(df['class'])
df['class'] = Y

### **Feature Correlation**
One of the most common method to discover the more meaningful features is generate a heatmap based on the features’ correlation. Mathematically the analysis is based upon the Pearson’s Correlation Coefficient:


\begin{equation}
ρ_(X,Y)=  \frac{(n(\sum{xy})-(\sum{x})(\sum{y}))}{\sqrt{[(n\sum{x})^2-((\sum{x}))^2][n\sum{y})^2-(\sum{y})^2]}}
\end{equation}


The numerator is the empirical estimate of the covariance of the jth feature and the target value, while the denominator is the square root of the variance of the jth feature, time the variance of the target. Basically then, this coefficient measures the strength of the relationship between two variables and their association with each other. To put it simply, the Pearson’s coefficient calculates the changes of a variable when another variable changes. 
Pearson’s Correlation ranges between -1 and 1, where in both cases it means there is a strong linear correlation, negative and positive respectively, between the two features. When the correlation coefficient is instead near zero, it means that the two features are not correlated at all, but this doesn’t mean it is a useless feature. On the contrary, when two features are strongly correlated, it is probable that one of them can be “discarded” because it does not provide any further information useful to the model to predict the target value. From this comes one of the most important statement in statistics "*Correlations does not imply causations*"


In [5]:
#@title
corr = df.corr(method='pearson')
cmap= ["#78290f","#ff7d00","#ffecd1","#15616d", "#202c39"]
fig = go.Figure(data=go.Heatmap(z=corr,x=df.columns, y=df.columns,colorscale=cmap))
fig.update_layout(
    title='Correlation between feautures',
    xaxis_nticks=len(df.columns),
    yaxis_nticks=len(df.columns),
    width = 700, height = 500)
fig.show()

Here is shown the correlation matrix of the feature, high positive correlated features are illustrated in blue, while highly negative correlated features are shown in orange.
As noticed in the pairplot, the fConc and fConc1 are strictly correlated. We now notice also a small correlation also bewteen fConc/fConc1 and fSize/fLength/fWidth. We can procede to reduce the the dimensionality of the dataset and extrapole some significant features. To do so in more structured manner, PCA for dimensionality reduction will be used. 


In [6]:
df = df.drop(['class'],axis=1)

In [7]:
df_train, df_test, y_train, y_test = train_test_split(df, Y,test_size=.25, random_state=0)
df_train

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
9168,25.9857,18.4585,2.5231,0.3538,0.1814,-25.4800,-6.2044,15.2170,56.1948,190.3300
8383,37.5265,21.7254,3.0988,0.2087,0.1087,37.1436,8.3872,8.8451,7.3742,141.9140
3980,58.8047,33.6055,3.5673,0.1798,0.0955,45.2378,56.4516,26.5973,4.6870,134.6540
8011,81.8663,22.5846,3.0037,0.2062,0.1076,-98.9128,56.3823,15.6502,4.4916,242.7150
9018,57.4159,17.4763,2.8344,0.2738,0.1428,22.4771,60.2797,12.2177,14.6429,202.6650
...,...,...,...,...,...,...,...,...,...,...
9225,34.7527,16.3799,2.6212,0.2703,0.1376,21.9455,23.6607,7.2786,16.5306,158.1200
13123,40.7440,16.0635,2.8145,0.3108,0.1668,15.8622,36.5069,-6.2138,13.3971,284.7751
9845,35.8286,16.8952,2.8802,0.3070,0.1746,38.7674,17.2509,11.3048,0.4720,234.8680
10799,20.0986,12.8671,2.4057,0.4558,0.2417,11.5039,9.7434,7.8750,21.6750,212.0980


In [ ]:
Y

array([0, 0, 0, ..., 1, 1, 1])

## *Standardization*

As mentioned before, features present very different distributions, magnitude, and units, so it is necessary to standardize them in order to make a reasonable analysis. In fact many algorithms based their analysis on these characteristics, in gradient-based algorithms having features all on similar scale can help the algorithm converging faster, or distance-based algorithms need also to have their features scaled so to give a fair importance to all of them. 
There are different ways to scale the features, in this work it is use the standardization, which is one of the most common techniques. The standardization is done by removing the mean (µ) of the features values and dividing by the standard deviation (σ) of the features values:
\begin{equation}
Z =  \frac{X- μ}{σ}
\end{equation}
In this way, the features result more comparable. We can see the effect of the standardization on the dataset in Figure 8. 


In [8]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(df_train)
df_train_scaled = pd.DataFrame(train_scaled, columns=df_train.columns)
df_train_scaled.describe()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist
count,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04,1.426500e+04
mean,-2.748123e-17,3.905432e-17,-2.589295e-16,-1.451248e-16,8.035400e-17,-1.288353e-17,2.120826e-17,-8.732354e-18,8.806291e-17,7.215671e-18
std,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00
min,-1.142097e+00,-1.192862e+00,-1.855999e+00,-2.005098e+00,-1.933050e+00,-7.464534e+00,-6.613837e+00,-7.833905e+00,-1.059394e+00,-2.574096e+00
25%,-6.778381e-01,-5.573448e-01,-7.335011e-01,-7.925623e-01,-7.824951e-01,-2.704419e-01,-4.446228e-01,-5.288635e-01,-8.478121e-01,-6.890751e-01
50%,-3.796533e-01,-2.739814e-01,-1.804014e-01,-1.467847e-01,-1.662869e-01,1.412262e-01,9.702134e-02,-2.131427e-03,-3.844270e-01,-2.650564e-02
75%,3.882435e-01,1.355662e-01,5.803490e-01,6.755599e-01,6.361332e-01,4.748606e-01,4.859354e-01,5.097069e-01,7.006551e-01,6.248484e-01
max,6.519952e+00,1.255787e+01,5.291116e+00,2.790005e+00,4.138175e+00,9.717173e+00,4.408578e+00,8.560659e+00,2.381585e+00,3.650962e+00


## **Outlier detection**

As anticipated in the Data Exploration section, the features present a high number of outliers, which are data points that stand out from the dataset and do not conform to the normal distribution of the whole dataset. These rare items can lead to errors and misclassification and depends on different factors: measurement errors, noise, novelty, etc. 
An easy way to visualize outliers in the data are boxplots. These graphs describe the data based on their quantiles and are not parametric, so no assumption is made upon the underling distribution. In particular they show five numbers: the maximum value, the minimum values, the sample median (or also second quantile), the first and the third quantile. The points that are not included between the maximum and minimum values are represented as dots, and they represent outliers. In Figure 10 we can see the boxplts of the features of our dataset, hued on their class value. 


In [ ]:
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}
px.box(df_train_scaled,color=y_train, title = "Box Plots Standardized Training Data",color_discrete_map=color_discrete_map)

Output hidden; open in https://colab.research.google.com to view.

As we can see, almost all the features present different univariate *outliers*. It is fun to notice that the features that have the greatest number of outliers corresponds to the ones with really hight level of kurtosis and skewness as observed in the Data exploration part.
To try to remove this abnormal data points, four different anomaly detection algorithms have been tested.


### IQR

IQR stands for Inter Quantile Range, and it is the difference between the third and first quartiles. Recalling that:
* Q1, the first quartile, is the 25% of the data, also denoted as 0.25 quantile or 25 percentile.
* Q3, the third quartile, is the 75% of the data, also denoted ad 0.75 quantile or 75 percentile.
* Q2, the second quartile or also the median, is the 50% of the data, 0.50 quantile or 50 percentile.

Therefore, the IQR is defined as:
\begin{equation}
IQR=Q3-Q1
\end{equation}
In order to detect outliers, a range is denoted by defining an upper and lower bound with the above-mentioned measures.
 \begin{equation}lower-bound=(Q1- 1.5*IQR) \end{equation}
 \begin{equation}upper-bound=(Q3-1.5*IQR)\end{equation}

All the points that are less the lower bound or more than the upper bound are considered outliers and therefore discarded. 


In [9]:
def outliers_iqr(X_train, y_train, stampa=False):
    Q1,Q3 = X_train.quantile(0.25), X_train.quantile(.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)
    outliers_index = []
    for i in range(len(X_train.columns)):
        out_lower = X_train.iloc[:,i] < lower_bound[i]
        outliers_index.append(out_lower[out_lower==True].index.values)
        out_upper = X_train.iloc[:,i] > upper_bound[i]
        outliers_index.append(out_upper[out_upper==True].index.values)
    outliers_index = np.concatenate(outliers_index, axis=0)
    outliers_index = np.unique(list(outliers_index.flatten())).tolist() ##elimino i duplicati 
    if stampa == True:
        print(f"With IQR there are {len(outliers_index)} outliers")
    to_keep =[i for i in X_train.index if i not in outliers_index]  
    X_train_iqr, y_train_iqr= X_train.iloc[to_keep, :], y_train[to_keep]
    px.box(X_train_iqr, color=y_train_iqr)

    return X_train_iqr, y_train_iqr

In [ ]:
X_train_iqr, y_train_iqr = outliers_iqr(df_train_scaled,y_train,stampa=True)
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}
px.box(X_train_iqr, color=y_train_iqr, title="Box Plots after IQR filter",color_discrete_map=color_discrete_map)

Output hidden; open in https://colab.research.google.com to view.

Using IQR Filter 2445 out of 1902 points have been identified as outliers. In fact, we can notice in the above graph that the y-range is been reduced a lot, which indicated that almost all the "*extreme*" univariate outliers have been removed.

### **Isolation Forest**



The Isolation Forest algorithm is a model-based method that isolates anomalies rather than profiles normal instances. It is based on two assumptions:
1.	The anomalies are a minority consisting of few instances
2.	They have attribute-values that are very “different” 

Therefore, the anomalies are expected to be isolated and closed to the root of a tree, called Isolation Tree. In fact, the Isolation Forest builds an ensemble of Isolation Trees on the datasets, the isolation of instances is done recursively until all instances have been isolated. The points which are considered outliers and therefore removed are those which have short average paths in the isolation trees, where the path length $h(x)$ is defined as the number of edges $x$ traverses an isolation tree from the root node to the external node.
Specifically, given a sample of data $X$ of $n$ instances, to build an isolation tree $X$ is divided by selecting randomly an attribute $q$ and a split value $p$ until either the tree reaches a height limit or $|X| = 1$ or all the data in $X$ have the same value. Note that the isolation tree is a perfect binary tree, so each node has zero or two daughter nodes. In this way, the total number of nodes is $2n-1$, and consequently the memory requirement is bounded and grows linearly with $n$. 
Each instance is assigned an anomaly score, depending on the average on the path length $h(x)$. Based on this score we can estimate if that instance is an anomaly or not:
* If the score is very close to 1, then the instance is definitely anomaly
* If the score is much smaller than 0.5 then the instance is almost surely not an anomaly
* If the score is ≈ 0.5 the entire sample does not really have any distinct anomaly.

In [10]:
def outliers_iso(X_train,y_train, stampa=False):
    iso = IsolationForest(contamination=0.1)
    yhat = iso.fit_predict(X_train)
    mask = yhat != -1
    X_train_iso, y_train_iso = X_train.iloc[mask, :], y_train[mask]
    if stampa == True:
        print(f"With Isolation forest there are {list(yhat).count(-1)} outliers")
    return X_train_iso, y_train_iso

In [ ]:
X_train_iso, y_train_iso = outliers_iso(df_train_scaled, y_train, stampa=True)
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}
px.box(X_train_iso, color=y_train_iso, title="Box plots after Isolation Forests",color_discrete_map=color_discrete_map)

Output hidden; open in https://colab.research.google.com to view.

In this case, the algorithm indentified fewer outliers (1522 out of 19020  datapoints) and in fact, the y-range in the Box-plots graph is wider. Still, many univariate outliers has been removed. 

### **Local Outlier Factor**



Local Outlier Factor (LOF) is an algorithm thought for outlier and anomaly detection, and it is the first algorithm to introduce also the concept of how much outlying a point is. In fact, it computes a score, called outlier factor, representing the degree of abnormality of the observation, basically tells us how likely the point is an outlier. <br/>
LOF “expands” the concept of outlier based on the assumption that for many real-world datasets the underlying structure is so complex that there must be considered other kinds of outliers. These objects are outlying relative to their local neighbourhood, in particular with respect to the density of their neighbourhood, and are called **local outliers**.
The term “local” refers to the fact that only a restricted neighbourhood of each point is taken into consideration.<br/>
Therefore, it is introduced a parameter $k$, which represents the number of neighbours considered. Using the right number for $k$ is not straightforward, with a too small number the algorithm looks only at nearby points, possibly missing out many noise points; on the other hand, a too large $k$ may miss local outliers.
This leads to the definition of $k-distance$ of an object $p$, denoted as $k-distance(p)$, which is the distance $d(p,o)$ between $p$ and an object $o \in D$ (Dataset), such that:

* For at least $k$ object $o’ \in D$, it holds that $d(o',p) ≤ d(o,p)$
* For at most $k-1$ object $o’ \in D$, it holds that $d(o',p) < d(o,p)$

For example, if $k$ was 3, the k-distance would be the distance of a point to the third closest point. <br/>
This leads to the concept of reachability distance of an object $p$ with respect to the object $o$:
<br/><br/>
\begin{equation}
reach-dist_k=max⁡{k-distance(o),d(p,o)}
\end{equation}
<br/>
Intuitively, if the object $p$ is really far away from object $o$, the reachability distance actually corresponds to the real distance of the two objects. This is basically a “smoothing effect” that can be controlled by the parameter $k$.<br/>
Linked to this, we have the definition of the local reachability density ($lrd$), which indicates basically the inverse of the average reachability distances based on the $k$ nearest neighbours of object $p$. Precisely,
<br/><br/>
\begin{equation} 
lrd=\frac{1}{\frac{\sum{reach-dist_k (p,o)}}{k}}
\end{equation}
<br/>
Finally, we arrive to the definition of the above-mentioned local outlier factor (LOF), formally,
<br/><br/>
\begin{equation} 
LOF_k (p)=  \frac{\frac{\sum{lrd(o)}}{lrd(p)}}{k}
\end{equation} 
<br/>
It is the average of the ratio of the local reachability density of p and those of $p$’s $k$-nearest neighbours. It is quite straightforward to see that the lower $p$’s reachability density is, the higher the reachability densities of $p$’s neighbours, and so the higher the $LOF$ coefficient. <br/>
In other words, the $LOF$ of a point tells the its density compares to those of its neighbours. If the density of a point is much lower than those of its neighbours, and therefore the $LOF >> 1$, then the point is far from a dense area and, hence, an outlier.

In [11]:
def outliers_lof(X_train,y_train, stampa=False):
    lof = LocalOutlierFactor(n_neighbors=400, novelty=True)
    lof.fit(X_train)
    outliers = lof.predict(X_train)
    mask_lof = outliers != -1
    if stampa==True:
        print(f"With LOF there are {list(outliers).count(-1)} outliers")
    X_train_lof, y_train_lof = X_train.iloc[mask_lof, :], y_train[mask_lof]
    return X_train_lof, y_train_lof


In [14]:
X_train_lof, y_train_lof = outliers_lof(df_train_scaled, y_train, stampa=True)
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}
px.box(X_train_lof, color=y_train_lof, title="Box plots after Local Outlier Factor", color_discrete_map=color_discrete_map)

Output hidden; open in https://colab.research.google.com to view.

This last algorithm indentifies the fewer number of outliers of all the others (787 out of 19020 ), in fact, the variables still presents a significant amount of abnormal points.

## **Dimensionality Reduction**

Dimensionality reduction refers to the process of taking the data in high dimensional space and map it in a lower dimensional space.  This helps avoiding the **curse of dimensionality**, which is a phenomena that appears in different domains such us machine learning, database, data mining. Dimensionality simply refers to the numeber of features/variables and it can become a problem when optimizing a model with a large number of variables. In fact, high dimensional functions are potentially much more complicated than low dimensional functions, which makes harder discern new samples.

The reduction is done by applying a linear transformation to the original data. The method used in this project, and probably the most common method used, is Principal Components Analysis (PCA). 


**PCA**<br/>
Let $x_i,… ,x_m$   be the original m vectors in $R^d$ , the purpose is to map $x$ with $y$ that is in a lower dimensional space $R^n$. So, we take $x ̃=Uy$ so that the $x ̃ $ is recovered version of $x$. Two matrixes are exploited to get the minimal distance between $x$ and $x ̃.$ Mathematically we aim at solving
\begin{equation}
\underset{W ∈R^{n,d},U∈R^{d,n}}{\mathrm{argmin}}\,\sum{i=1}^{m}‖x_i-UWx_i ‖_2^2  
\end{equation} 
It can be proved that this formula is equal to the below formula:
\begin{equation}
\underset{U ∈R^{d,n}:U^TU=1}{\mathrm{argmin}}\,trace(U^T \sum{i=1}^{m}x_i x_i^TU)
\end{equation}
Where the trace operator is the sum of the diagonal entries of the matrix. In this way it can be found the solution of the PCA optimization problem by setting the U matrix to have the columns $u_1,…,u_n$, which are the n largest eigenvectors  of the matrix $A=\sum{i=1}^{m}x_i x_i^T$, and set $W=U^T$. 
It is usually a standard practice to centre the data before applying PCA, this is related to the concept of maximal variance. Variance in the data is really important, because it corresponds to information. Thanks to the variance we can understand if a feature is more important than another.
In simpler words, the PCA optimization problem explained before corresponds in choosing the “best line” on which project the original data to get the minimal error, this “best line” corresponds to the one in the direction of the maximum variance. The largest variance is given by the eigenvector corresponding to the largest eigenvalue of the matrix A.
The principal components found by PCA are all linear combination of the features. The first principal component has the largest variance:
$Z_1= ϕ_{11} X_1+ ϕ_{21} X_2+⋯+ ϕ_{m1} X_m$
And it is a normalized linear combination, which means that the $\sum{j=1}^{m}ϕ_{j1}^2=1$
The values $ϕ_{j1},…,ϕ_{jm}$ are called principal components loadings. Projecting the data point $x_1,…,x_m$ onto these directions, we get the principal component scores $z_1,…,z_m$. Note that the $j$ principal components must be uncorrelated with each other, which is equivalent to constraining their directions to be orthonormal. 
In this way, PCA can be thought of as finding the new orthonormal basis by rotating the old axis until the direction of the maximal variance is found.


In [15]:
# Pareto chart to see how many feature a need for a good result ---> circa 6
pca = PCA()
pca.fit(df_train_scaled)
pca.explained_variance_ratio_
reduced_df_train = pca.transform(df_train_scaled)
color_discrete_map = {0:'#15616d', 1: '#ff7d00'}

total_var = pca.explained_variance_ratio_.sum() * 100
fig = go.Figure(data=go.Bar(x=np.arange(1,29), y=pca.explained_variance_ratio_, name='Variance Explained',marker_color='#004052', marker_line_color='#004052',
                  marker_line_width=1.5, opacity=0.7))
fig.add_trace(go.Scatter(x=np.arange(1,29), y=pca.explained_variance_ratio_.cumsum(), name='Cumulative',mode="lines+markers",marker=dict(color='#dc562e')))

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

*How to understan how many principal components to chose?* One of the most frequent method to use is explointing the bias-variance trade-off. To better visualize the variance explanied the Pareto diagram has been used. On the graph, the blue histogram represents the quantity of variance explained by every single possible component, while the red scattered line is the cumulative explained variance. The rule of thumbs tells me to retain a number of principals components so that the variance mainted is bewteen 70% and 90%. So I decided to maintain 5 principal components.


In [16]:
pca = PCA(5)
pca.fit(df_train_scaled)
X_new = pca.transform(df_train_scaled)
#X_val_new = pca.transform(X_val_clean)

# **Unbalanced dataset**

We talk about unbalanced dataset when a class has the majority of the prediction with respect to the other. This leads the models to predict more often the majority class, which is usually the less important, resulting in  lower values of accuracy. <br/>
High unbalanced dataset are common when dealing with real data such us telecommunication customers, radar images, text classification and so on. In the case of the analysed dataset the imbalance is due to the fact that re-creating gamma rays for the simulation is easier than recreating hadrons. While in reality, the majority class would have been hadron (h -> 1).
During the year different methods to deal with the this situation have been proposed both at data level and at algorithm levels. The former consists of all the forms of re-sampling data like random oversampling with replacement, random undersampling, direct oversampling and direct undersampling. While the algorithm levels method focus more theorethical reasonings, like for istance adjusting the probability estimate of each class, or adjusting the decision threshold. 

<a href="https://www.researchgate.net/publication/228084509_Handling_imbalanced_datasets_A_review">[3]</a>

## **Random Undersampling**

Random undersampling is a non-heuristic methond that aims at eliminating samples from the majority class to balance the class distribution. The idea is to overcame the normal behaviour of machine learning algoriths that tends to preferred the majority class especially in presence of uncertainty. The main drawback of this method is that it can potentially discard important information for the discrimination process.
The idea behind this implementation of a consistent subset is to
eliminate the examples from the majority class that are distant from the decision border, since these sorts of examples might be considered less relevant for learning.

<p align="center"><img src="https://dataaspirant.com/wp-content/uploads/2020/08/17-undersampling.png" width=500 height=300/></p>


In [ ]:
rus = RandomUnderSampler() 
X_train_rus, y_train_rus = rus.fit_resample(X_new, y_train)

In [ ]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("Random Under Sampling", "Original"))
fig.add_trace(go.Bar(y=[len(y_train_rus[y_train_rus==0]), len(y_train_rus[y_train_rus==1])],x=['gamma', 'hadron'],marker_color='#004052'), 1,1)
fig.add_trace(go.Bar(y=[len(y_train[y_train==0]), len(y_train[y_train==1])],x=['gamma', 'hadron'],marker_color='#dc562e'),1,2)
fig.update_layout(height=400, width= 1000,showlegend=False)
fig.show()

As we can see, the total numer of instances for each class is now equal to the number of instances of the previous minority class.

## **SMOTE**

Regarding the algorithm level the algorithm we analyze Synthetic Minority Oversampling Technique (SMOTE). The minority class is over-sampled
by taking each minority class sample and introducing synthetic examples along the line segments joining any/all of the k minority class nearest neighbors. These synthetic samples are generated by taking the difference between the feature vector (sample) under consideration and its nearest neighbor; then multipling this difference by a random number between 0 and 1, and adding it to the feature vector under consideration. In this way, the selection of a random point is along the line segment between two specific features.
This approach effectively forces the decision region of the minority class to become more general.
<p align="center"><img src="https://editor.analyticsvidhya.com/uploads/77417image1.png" width=500 height=300/></p>

<a href="https://arxiv.org/pdf/1106.1813.pdf">source</a>

In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_new, y_train)

In [ ]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("SMOTE", "Original"))
fig.add_trace(go.Bar(y=[len(y_train_smote[y_train_smote==0]), len(y_train_smote[y_train_smote==1])],x=['gamma', 'hadron'],marker_color='#004052'), 1,1)
fig.add_trace(go.Bar(y=[len(y_train[y_train==0]), len(y_train[y_train==1])],x=['gamma', 'hadron'],marker_color='#dc562e'),1,2)
fig.update_layout(height=400, width= 1000,showlegend=False)
fig.show()

Indeed after SMOTE we now have the same number of istancens of the previous majority class.

# **Training and Testing**

### **K-Fold Cross Validation**

K-fold validation is a re-sampling procedure widely used in Machine learning. It consists of dividing the data into k different sets to select the best performing “model”, where iteratively each fold is used as validation set, for example in the first iteration the first fold is used as validation set and all the others as training set

For each group of observation an evaluation metric is computed, so we end up with k evaluations. The k-Fold CV is then computed by averaging these values:
\begin{equation}
CV_K=   \sum_{k=1}^{K}{\frac{n_k}{n} Err_k} 
\end{equation}

where $Err_k = \sum_{i\in{C_k}}\frac{I(y_i \neq \hat{y_i})}{n_k} $
<br/><br/>
<p align="center"><img src="https://miro.medium.com/max/1688/0*yRHMkhX-tlcM8qAR.gif" width="500" height="300"/></p>


Then, we evaluate the analysis done through the training phase on the totally unseen data of the test set. Also in this case a function has been created in which the steps of the "pipeline" are insterted in such a way that each algorithm (Standard Scalar, Outlier detection, PCA, inference) has been trained only on the training set and evalutate on the test set.

**Stratified K-Fold**
<br/>Stratified Kfold is a variation that takes into consideration the unbalance of the original dataset, each set of indexes is created so that it contains approximately the same percentage of samples classes as the original dataset. This method is reccommanted when dealing with large unbalanced dataset, and even if this dataset the difference is not particurarly big, I decided to used it over the normal KFold.

### **Evaluation Metrics**

In this work we are facing a binary classification problem, so the two errors possible are false-positive (FP) and false-negative (FN). In these cases, we mis-classify respectively a prediction y = 1, when the real value is 0, and a prediction y=0, when the real value is 1. One of the principal ways to represent these values is called Confusion Matrix (aka error matrix), in which each row is an instance of the predictive class and each column are the instances of the true class.
<p align="center"><img src="https://miro.medium.com/max/2102/1*fxiTNIgOyvAombPJx5KGeA.png" width="300" height="200"/></p>
As we can visually see in Figure 9, the diagonal values on the matrix represents the correct classification, while the off-diagonal values are the mis-classified values.
From the table, we can also compute the metrics to evaluate the classification models:

* Accuracy: probably the simplest metric is defined as the number of correct predictions divided by the total number of predictions. 
\begin{equation}
Accuracy=  \frac{TP+TN}{TP+TN+FP+FN}
\end{equation}
* Recall or True Positive Rate (TPR), or if you are dealing with biomedical features, Sensitivity, defined as the number of correctly positive predicted values over the total actual positives.
\begin{equation}
 Recall =  \frac{TP}{TP+FN}
 \end{equation}
* Precision or Positive Predictive Value: is defined as the number of true predicted positives over the total number of positives. 
\begin{equation}
Precision=  \frac{TP}{TP+FP}
\end{equation}
* F-1Score: is a function of both precision and recall. 
\begin{equation}
F1-Score=\frac{2*Sesitivity*Precision}{Sensitivity+Precision}
\end{equation}

All the previous pre-processing steps are inserted in the functions training and test, so that the trasformation are applied taking into consideration points belonging excusely to the training set, and the validation and test are completely new data points for the model to analyse.

In [17]:
#@title
def training(algorithm, param_grid,outliers_detect,balancing_algo,doPCA=True, outliers=True,balancing=True):
    kf = StratifiedKFold(n_splits=5,shuffle=True ,random_state=42)

    total_results = []

    for params in ParameterGrid(param_grid):
        mean_acc = []
        mean_rec = []
        mean_prec = []
        mean_f1 = []
        for train_index, test_index in kf.split(df_train, y_train):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_val = df_train.iloc[train_index,:], df_train.iloc[test_index,:]
            Y_train, y_val = y_train[train_index], y_train[test_index]

            #Standardizing fitting only on the training set
            std = StandardScaler()
            std.fit(X_train)
            X_train = std.transform(X_train)
            X_val = std.transform(X_val)

            X_train = pd.DataFrame(X_train, columns=df_train.columns)
            X_val = pd.DataFrame(X_val, columns=df_test.columns)

            if outliers:
                ## Delete outliers
                X_train_clean, y_train_clean= outliers_detect(X_train,Y_train,stampa=False)
            else:
                X_train_clean,y_train_clean = X_train, Y_train

            if doPCA:
                ## PCA
                pca = PCA(5)
                pca.fit(X_train_clean)
                X_train_pca = pca.transform(X_train_clean)
                X_val_pca = pca.transform(X_val)
            else:
                X_train_pca = X_train_clean
                X_val_pca = X_val

            if balancing:
                ##SMOTE
                X_train_bal,y_train_bal = balancing_algo.fit_resample(X_train_pca, y_train_clean)
            else:
                 X_train_bal,y_train_bal = X_train_pca, y_train_clean


            ## Training model
            algo = algorithm(**params)
            algo.fit(X_train_bal,y_train_bal)
            y_pred = algo.predict(X_val_pca)

            ## Evaluate
            accuracy = accuracy_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred)
            f1 = f1_score(y_val, y_pred)

            mean_acc.append(accuracy)
            mean_rec.append(recall)
            mean_prec.append(precision)
            mean_f1.append(f1)

        data = [params, np.mean(mean_acc), np.mean(mean_rec), np.mean(mean_prec), np.mean(mean_f1)]
        total_results.append(data)

    print("Done!")
    return total_results, algo


In [18]:
#@title
def testing(algorithm, best_param,outliers_detect,balancing_algo,doPCA=True,outliers=True,balancing=True):

    algo = algorithm(**best_param)

    #Standardizing fitting only on the training set
    std = StandardScaler()
    std.fit(df_train)
    X_train = std.transform(df_train)
    X_test = std.transform(df_test)

    X_train = pd.DataFrame(X_train, columns=df_train.columns)
    X_test = pd.DataFrame(X_test, columns=df_test.columns)
   
    if outliers:
        ## Delete outliers
        X_train_clean, y_train_clean= outliers_detect(X_train,y_train,stampa=False)
    else:
        X_train_clean,y_train_clean = X_train, y_train

    if doPCA:
        ## PCA
        pca = PCA(5)
        pca.fit(X_train_clean)
        X_train_pca = pca.transform(X_train_clean)
        X_test_pca = pca.transform(X_test)
    else:
        X_train_pca = X_train_clean
        X_test_pca = X_test
    if balancing:
        ##SMOTE
        X_train_bal,y_train_bal = balancing_algo.fit_resample(X_train_pca, y_train_clean)
    else:
        X_train_bal,y_train_bal = X_train_pca, y_train_clean


    algo.fit(X_train_bal,y_train_bal)

    y_pred = algo.predict(X_test_pca)
    print(classification_report(y_test, y_pred))

    #Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    arr_val = [accuracy, recall, precision, f1]

    cf_matrix = confusion_matrix(y_test, y_pred)
    return cf_matrix, arr_val

In [19]:
 #@title
 def print_conf_acc(pd_est):   
    df = pd.DataFrame(pd_est, columns=["params","acc","rec","prec","f1"])
    alias = [f'Configuration {i+1}' for i in range(len(pd_est))]
    fig = make_subplots(rows=1, cols=1, subplot_titles="StratifiedKFolds configuration")
    fig.add_trace(go.Bar(x=alias,y=df['acc'],name="Accuracy", marker_color=("#78290f")))
    fig.add_trace(go.Bar(x=alias,y=df['rec'],name="Recall", marker_color=("#ff7d00")))
    fig.add_trace(go.Bar(x=alias,y=df['prec'],name="Precision", marker_color=("#202c39")))
    fig.add_trace(go.Bar(x=alias,y=df['f1'],name="F1-score",marker_color=("#15616d")))
    fig.show()
    return df

In [73]:
#@title
def print_lc(estimator, learning_curves = True):

    fig = make_subplots(rows=1, cols=1, subplot_titles=["Confusion Matrix","Learning Curves"])

    ## Learning Curves
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    train_sizes, train_scores, test_scores = learning_curve(estimator,df_train,y_train, cv=cv, scoring='accuracy',shuffle=True)
    #train_scores_mean = 1-np.mean(train_scores,axis=1)#converting the accuracy score to misclassification rate
    #test_scores_mean = 1-np.mean(test_scores,axis=1)#converting the accuracy score to misclassification rate
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    upper_bound = train_scores_mean+train_scores_std
    lower_bound = train_scores_mean-train_scores_std
    fig.add_trace(go.Scatter(x=train_sizes,y=train_scores_mean,name="Training accuracy",marker_color=cmap[1]))
    fig.add_trace(go.Scatter(x=train_sizes,y=upper_bound,   line=dict(width=0),
            mode='lines',fill='tonexty',fillcolor='rgba(254, 200, 154, 0.2)',showlegend=False, hoverinfo="skip"))
    fig.add_trace(go.Scatter(x=train_sizes,y=lower_bound,   line=dict(width=0),
            mode='lines',fill='tonexty', fillcolor='rgba(254, 200, 154, 0.2)',showlegend=False, hoverinfo="skip"))
    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean,name="Test accuracy", marker_color=cmap[3]))
    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean+test_scores_std,   line=dict(width=0),
            mode='lines',fill='tonexty',fillcolor='rgba(0,100,80,0.1)',showlegend=False, hoverinfo="skip"))
    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean-test_scores_std,   line=dict(width=0),
            mode='lines',fill='tonexty', fillcolor='rgba(0,100,80,0.2)',showlegend=False, hoverinfo="skip"))
    
    fig.update_layout(autosize=False,
    width=600,
    height=400,
    xaxis_title='Training Example',
    yaxis_title='Scores')
    
    
    fig.show()


def print_cfs(arr_cfs):

    if len(arr_cfs) != 5:
        raise RuntimeError("Wrong number of cfs")
    
    cf1, cf2, cf3, cf4, cf5 = arr_cfs

    fig = make_subplots(rows=3, cols=2, subplot_titles=[cf1[1],cf2[1],cf5[1],cf3[1],cf4[1]])

    # CF1
    trace_cf = ff.create_annotated_heatmap(z = cf1[0],x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
    fig.add_trace(go.Heatmap(trace_cf.data[0]),row=1,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str( cf1[0][0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str( cf1[0][1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str( cf1[0][0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str( cf1[0][1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=1)
    
    # CF2
    trace_cf = ff.create_annotated_heatmap(z = cf2[0],x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
    fig.add_trace(go.Heatmap(trace_cf.data[0]),row=1,col=2)
    fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str(cf2[0][0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=2)
    fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str(cf2[0][1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=2)
    fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str(cf2[0][0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=2)
    fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str(cf2[0][1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=1,col=2)

    # CF3
    trace_cf = ff.create_annotated_heatmap(z = cf3[0],x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
    fig.add_trace(go.Heatmap(trace_cf.data[0]),row=2,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str(cf3[0][0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=3,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str(cf3[0][1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=3,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str(cf3[0][0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=3,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str(cf3[0][1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=3,col=1)
    
    # CF4
    trace_cf = ff.create_annotated_heatmap(z = cf4[0],x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
    fig.add_trace(go.Heatmap(trace_cf.data[0]),row=2,col=2)
    fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str(cf4[0][0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=2)
    fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str(cf4[0][1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=2)
    fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str(cf4[0][0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=2)
    fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str(cf4[0][1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=2)
    
    # CF5
    trace_cf = ff.create_annotated_heatmap(z = cf5[0],x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
    fig.add_trace(go.Heatmap(trace_cf.data[0]),row=3,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str(cf5[0][0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=1)
    fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str(cf5[0][1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str(cf5[0][0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=1)
    fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str(cf5[0][1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""), row=2,col=1)
    
     
    fig.update_layout(autosize=False,
    width=900,
    height=650,
    xaxis_title='Predicted',
    yaxis_title='Ground Truth')
    
    fig.show()



In [21]:
#@title
def chose_best_params(arr_params):
    indexes = []
    df = pd.DataFrame(arr_params, columns=["params","acc","rec","prec","f1"])
    alias = [f'Configuration {i+1}' for i in range(len(arr_params))] 
    indexes.append(df['acc'].idxmax())
    indexes.append(df['rec'].idxmax())
    indexes.append(df['prec'].idxmax())
    indexes.append(df['f1'].idxmax())

    #best = mode(indexes)
    c = Counter(indexes)
    best,_ = c.most_common(1)[0]
    print(f"The best configuration is {alias[best]}: {df['params'][best]}")
    return df['params'][best]

## **The Bias-Variance Trade-off**


The Bias-Complexity trade off is related to the chose of a good hypothesis class for our predictors. The trade off is needed because on one hand we want a class that we belivie it contains the hypothesis with the smallest possible error, but on the other hand we cannot simply choose the richest class possible (i.e. a class that consists of all possible functions over the given domain).
In order to resolve this trade off the we decompose the Empirical Risk in two:
* *The Approximation Error*: is the minimum risk that can be achived by the hypothesis class. It does not depends on the sample size but on the hypothesis class chosen. So increasing the class can reduce the the approximation error.
* *The Estimation Error*: is the difference between the approximation error and error achieved by the ERM estimator. This error is linked to the fact the the Empirical Risk (i.e. the training error) is an estimation of the true risk, and the quality of this estimation depends on the sample size and the size or complexity of the hypothesis class. <br/>

This lead to the tradeoff since on the one hand we want to increase the hypothesis class in order to reduce the approximation error but this at the same time may lead to an increase in the estimation error (since a richer hypothesis class may lead to overfitting). On the other hand deciding to use a small hypothesis class reduces the estimations error but may increase the approximation error, resulting in underfitting. 

One of the common thing to do for understanding which of the two cases of error the algorithm is suffering is plotting *learning curves*. In this "reppresentation" the algorithm is trained on increasing part of the training set, starting for example with 10% of the whole dataset, then with the 20% and so on. At each steps the training error is calculated on both training and validation set. In this way if the trend of the training error and the validation error remains constant for all the steps, then the algorithm is suffering of approximation error since it is not actually learning more, while if the validation error starts as a constant and then starts decreasing we are in the estimation error scenario

<p align="center"><img src="https://www.baeldung.com/wp-content/uploads/sites/4/2020/07/fitgraph.jpg" width="400" height="250"/></p>


In the case of this project, it is used the method "learnin_curve" provided by the library Sklearn, which uses the score/accuracy on the training and validation sets, so the higher the values the better.

## **Algorithms**

The algorithms chosen to inference the dataset are 4 and they are tuned and validated with Stratified 5-Fold cross validation method. Moreover, a custom pipeline is created in a fuction to train the different algorithms (PCA, Outlier detection and undersampling/SMOTE) only on the training set and not on the validation set, which instead reflects a proper test set with totally unseen data. 


### **Random Forest**
Random Forest is a classifier based on an ensemble of decision trees. A decision tree is a simple and intuitive method which can be used both for classification and regression. Each internal node of the tree $v$ contains a logical condition that divides the features space $X$ into regions/subsets and associated with each leaf node there is a partition of $X$ and a regional prediction function, that in a classification setting corresponds to the set of possible classes (in this case just two, gamma or hadron). It is very common to construct the tree with a greedy top-down approach, which means that the tree is built gradually starting from the root node and locally optimal decisions are made at the construction of each node. These decisions are based on a splitting rule which aim to maximize the “gain” of the algorithm, in fact, among all the possible splits it is chosen the one that provides the maximum “gain” or it is decided to not split further and this will be a leaf node.  There are different solutions to define this “gain”, we can analyse two:
* Information Gain: is the difference between the entropy before and after the splitting.
\begin{equation}
Entropy= \sum_{i=1}^{C}-p_i*\log_{2}{p_i}
\end{equation}
* Gini Index: 
\begin{equation}
Gini= 1- \sum_{i=1}^{C}p_i^2 
\end{equation}

Both these measures are a level of “impurity” of the node, so the lower they are the better.
The purpose of Random Forest is to limit the risk of overfitting in a single decision tree. In fact, decision tree (especially when they are deep) heavily rely on their training set and even a small change in the data can totally revolutionize the results. The major idea behind Random Forest is bootstrap aggregation sampling, or also called **bagging**, which combine multiple prediction functions learned from different data sets. In the case of Random forest, $T_1,…T_B$, sets are sampled from the training set to create $B$ separate models. Then, using a majority voting criterium, the most frequent class among the $B$ predictors is chosen. In case of a regression setting, the prediction function is often chosen as the mean response.


In [22]:
total_results = pd.DataFrame([], columns=['Algorithm','Configuration','Accuracy', 'Precision','Recall','F1-score'])

In [23]:
params_rdmf = { "random_state": [0],
                "criterion": ["entropy","gini"],
                "n_estimators": [20,50,100,200],
               }

pd_rnd, rnd = training(RandomForestClassifier, params_rdmf, outliers_iso, SMOTE())

Done!


In [24]:
df_rndforest = print_conf_acc(pd_rnd)

The best configuration is chosen comparing the values of each metric for each configuration. The configuration which has the most high score is decided to be the best. 

In [25]:
best_rnd = chose_best_params(pd_rnd)

The best configuration is Configuration 2: {'criterion': 'entropy', 'n_estimators': 50, 'random_state': 0}


In [75]:
rnd_cfs = []
cf_matrix_rnd_1, test_rnd_iso_pca_smote = testing(RandomForestClassifier, best_rnd,outliers_iso, SMOTE()) 
rnd_cfs.append((cf_matrix_rnd_1,"Test Random Forest with Iso, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Random Forest',"Test Random Forest with Iso, PCA, SMOTE",
                                test_rnd_iso_pca_smote[0], test_rnd_iso_pca_smote[1],
                                test_rnd_iso_pca_smote[2],test_rnd_iso_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_rnd_2, test_rndiso_pca_rus = testing(RandomForestClassifier, best_rnd,outliers_iso, RandomUnderSampler()) 
rnd_cfs.append((cf_matrix_rnd_2,"Test Random Forest with Iso, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Random Forest',"Test Random Forest with Iso, PCA, Random Undersampling",
                                test_rndiso_pca_rus[0], test_rndiso_pca_rus[1],
                                test_rndiso_pca_rus[2], test_rndiso_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_rnd_3, test_rnd_lof_pca_smote = testing(RandomForestClassifier,best_rnd,outliers_lof, SMOTE()) 
rnd_cfs.append((cf_matrix_rnd_3,"Test Random Forest with LOF, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Random Forest',"Test Random Forest with LOF, PCA, SMOTE",
                                test_rnd_lof_pca_smote[0], test_rnd_lof_pca_smote[1],
                                test_rnd_lof_pca_smote[2],test_rnd_lof_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_rnd_4, test_rnd_lof_pca_rus = testing(RandomForestClassifier, best_rnd,outliers_lof, RandomUnderSampler()) 
rnd_cfs.append((cf_matrix_rnd_4,"Test Random Forest with LOF, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Random Forest',"Test Random Forest with LOF, PCA, Random Undersampling",
                                test_rnd_lof_pca_rus[0],test_rnd_lof_pca_rus[1],
                                test_rnd_lof_pca_rus[2],test_rnd_lof_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_rnd_5, test_rnd_no_precosseing = testing(RandomForestClassifier, best_rnd,outliers_lof, RandomUnderSampler(), doPCA=False,outliers=False, balancing=False) 
rnd_cfs.append((cf_matrix_rnd_5,"Test Random Forest with no preprocessing"))
total_results = total_results.append(pd.Series(['Random Forest',"Test Random Forest with no preprocessing",
                                test_rnd_no_precosseing[0],test_rnd_no_precosseing[1],
                                test_rnd_no_precosseing[2],test_rnd_no_precosseing[3]],index=total_results.columns),ignore_index=True)

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      3079
           1       0.68      0.74      0.71      1676

    accuracy                           0.79      4755
   macro avg       0.77      0.78      0.77      4755
weighted avg       0.79      0.79      0.79      4755

              precision    recall  f1-score   support

           0       0.87      0.77      0.82      3079
           1       0.65      0.79      0.71      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.78      0.76      4755
weighted avg       0.79      0.78      0.78      4755

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      3079
           1       0.72      0.72      0.72      1676

    accuracy                           0.80      4755
   macro avg       0.78      0.78      0.78      4755
weighted avg       0.80      0.80      0.80      4755

              preci

In [69]:
bla_cf, bla_bla = testing(RandomForestClassifier,{'n_estimators':100, 'criterion':'entropy', 'random_state':0} ,outliers_iso, SMOTE(),doPCA=False,outliers=False, balancing=False) 


              precision    recall  f1-score   support

           0       0.88      0.95      0.92      3079
           1       0.90      0.76      0.82      1676

    accuracy                           0.89      4755
   macro avg       0.89      0.86      0.87      4755
weighted avg       0.89      0.89      0.88      4755



In [49]:
fig = make_subplots(rows=1, cols=1)
trace_cf = ff.create_annotated_heatmap(z = bla_cf,x = ["0","1"],y = ["1","0"],showscale  = False,colorscale=cmap)
fig.add_trace(go.Heatmap(trace_cf.data[0]))
fig.add_trace(go.Scatter(x=[0],y=[0],mode="text",text=[str(bla_cf[0][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""))
fig.add_trace(go.Scatter(x=[0],y=[1],mode="text",text=[str(bla_cf[1][0])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""))
fig.add_trace(go.Scatter(x=[1],y=[0],mode="text",text=[str(bla_cf[0][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""))
fig.add_trace(go.Scatter(x=[1],y=[1],mode="text",text=[str(bla_cf[1][1])],textfont=dict(family="sans serif",size=18,color="#ffffff"),name=""))
fig.update_layout(height=400, width= 400,showlegend=False, title="Confusion matrix Random Forest")
fig.show()

In [29]:
total_results

,Algorithm,Configuration,Accuracy,Precision,Recall,F1-score
0,Random Forest,"Test Random Forest with Iso, PCA, SMOTE",0.791588,0.738663,0.691234,0.714162
1,Random Forest,"Test Random Forest with Iso, PCA, Random Under...",0.774763,0.780430,0.650423,0.709520
2,Random Forest,"Test Random Forest with LOF, PCA, SMOTE",0.794953,0.708831,0.709254,0.709042
3,Random Forest,"Test Random Forest with LOF, PCA, Random Under...",0.785699,0.771480,0.670295,0.717337
4,Random Forest,Test Random Forest with no preprocessing,0.885594,0.764320,0.895804,0.824855


In [74]:
print_cfs(rnd_cfs)

In [31]:
print_lc(RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=0, bootstrap=True,))

Notice that accuracy is always higher than the other metrics, this is probably due to the imbalace of the two classes in the dataset. In fact the model is better at predicting class 'g'(0). This is easy to get by looking at the confusion matrixes, where the number of misclassification of the class 1 are always quite high. Notice also that the model without any kind of preprocessing has a higher accuracy but the number of false positive for the class h (1) is definetelly much higher than the other configurations. Looking instead at the learning curves, the training trend is almost a straight line, this means that the algorithm is suffering of quite high variance hence a severe overfitting.

### **Support Vector Machine**



Support Vector Machine is a supervised method used both for classification and regression. The main idea is to find and hyperplane that is able to distinctly classifies the data. We say that a training set $S$ is linearly separable if 
<br/><br/>
\begin{equation}
∀i ∈[m],y_i (<w,x_i> +b )>0
\end{equation}
<br/>
For any separable training set there are a lot of possible hyperplanes, it is taken the ones that maximize the margin, i.e. the minimal distance between a point of the training set and the hyperplane itself. In this way, the formal rule of the so called *hard-SVM* is
<br/><br/>
\begin{equation}
argmax_{(w,b):||w||=1}⁡min_{i ∈[m]}|⁡|<w,x_{i}>+b|| \quad\quad     s.t.∀i,y_i  (<w,x_{i}>+b)>0 
\end{equation}
<br/>
In other words, hard-SVM searches for $w$ of minimal norm among all vectors that separate the data and for which $|<w,x_i>+b|≥1$ for all $i$. 
The solution of the Hard-SVM is therefore a solution to the above formula:
<br/><br/>
\begin{equation}
w ̂= \frac{w_0}{||w_0 ||}  \quad\quad\quad\quad    b ̂=\frac{b_0}{||w_0 ||}
\end{equation}
<br/>
This solution is supported by the points that are exactly at distance $1/|(|w_0 |)|$ from the separating hyperplane. These vectors are called *Support Vectors*, hence the name of the algorithm. <br/>
In particular, let $I={ i∶|<w_0,x_i>|=1}$, there exist coefficients $α_1,…,α_m$ such that $w_0 = \sum_{i ∈I}α_i x_i$.<br/>
The examples ${x_i ∶i ∈I}$ are the support vectors.

The problem of the Hard-SVM is that it assumes data is linearly separable, which is a quite strong assumption. Thus, a relaxation of this algorithm comes to help, called *Soft-SVM*, and it can be applied even when data is not linearly separable. This relaxation consists of allowing the constraint to be violated for some samples. This is done with the introduction of *slack variables* $ξ_1,…,ξ_m$ in the constraint:
<br/><br/>
\begin{equation}
|<w,x_i>+b|≥1- ξ_i
\end{equation}
<br/>
In this way $ξ_i$ measures how much the constraint is being violated. Moreover, it holds that: 
\begin{equation}
\sum_{i=1}^{m}ξ_i  ≤C 
\end{equation}
<br/>
Where C is a non-negative tuning parameter, also known as *Regularization parameter*, because it basically determines the number of violations we want to allow. If $C$ is small, the margin is small and therefore we allow rarely “mistakes”, on the other hand, if $C$ is large, the margin is wider, and we allow more “mistakes”.  



**The Kernel Trick**

This is a “trick” used when data cannot be linearly separated. To solve the problem a non-linear mapping $ψ$ is applied to map the instances in a higher dimensional space and the learning of the feature space happens in this space. However, compute linear separators in very high dimensional space can be very computation expensive, so it is exploited a kernel that (in this context) refers to the inner product in the feature space. Formally, given an embedding $ψ$ of some domain $X$, we define the kernel function as 
<br/><br/>
\begin{equation}
K = (x,x') = <\psi(x),\psi(x')>
\end{equation}
<br/>
The Kernel can be though as specifying a similarity between the instances and the embeddings realised as inner products. Thus, the “success” of the algorithm depends on choosing a good mapping $ψ$, that is able to separate the data in the features space. There are different kernels, in this project we tested the liner-kernel, which actually corresponds to the soft-SVM, and the *RBF (Radis Basis Function)* kernel:
<br/><br/>
\begin{equation} 
K(x,x') = \exp(-\gamma\sum_{i=1}^m(x_i,x_{i}^{'})^2)
\end{equation}
<br/>
This kernel basically calculates the distances of the data point from the margin with a sort of Gaussian shape, and the parameter $γ$, which is a positive constant, measures how “quick” this distance increases.

In [ ]:
params_svm = {"kernel": ["linear","rbf"],
              "C": [1,10,100]}

pd_svc, svc = training(SVC, params_svm, outliers_iso,SMOTE())

Done!


In [ ]:
df_svm = print_conf_acc(pd_svc)

SVM presents configuration which are more different between each other with respect to the previous algorithm. In particular, configuration 2 seems has parcularly higher recall than all the the other.

In [ ]:
best_svc = chose_best_params(pd_svc)

The best configuration is Configuration 4: {'C': 10, 'kernel': 'rbf'}


In [ ]:
svc_cfs = []
cf_matrix_svc_1, test_svc_iso_pca_smote = testing(SVC, best_svc,outliers_iso, SMOTE()) 
svc_cfs.append((cf_matrix_svc_1,"Test SVC with Iso, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Support Vector Machines',"Test SVC with Iso, PCA, SMOTE",
                                test_svc_iso_pca_smote[0],test_svc_iso_pca_smote[1],
                                test_svc_iso_pca_smote[2],test_svc_iso_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_svc_2, test_svc_iso_pca_rus = testing(SVC, best_svc,outliers_iso, RandomUnderSampler()) 
svc_cfs.append((cf_matrix_svc_2,"Test SVC with Iso, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Support Vector Machines',"Test SVC with Iso, PCA, Random Undersampling",
                                test_svc_iso_pca_rus[0],test_svc_iso_pca_rus[1],
                                test_svc_iso_pca_rus[2],test_svc_iso_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_svc_3, test_svc_lof_pca_smote = testing(SVC,best_svc,outliers_lof, SMOTE()) 
svc_cfs.append((cf_matrix_svc_3,"Test SVC with LOF, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Support Vector Machines',"Test SVC with LOF, PCA, SMOTE",
                                test_svc_lof_pca_smote[0],test_svc_lof_pca_smote[1],
                                test_svc_lof_pca_smote[2],test_svc_lof_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_svc_4, test_svc_lof_pca_rus = testing(SVC, best_svc,outliers_lof, RandomUnderSampler()) 
svc_cfs.append((cf_matrix_svc_4,"Test SVC with LOF, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Support Vector Machines',"Test SVC with LOF, PCA, Random Undersampling",
                                test_svc_lof_pca_rus[0],test_svc_lof_pca_rus[1],
                                test_svc_lof_pca_rus[2],test_svc_lof_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_svc_5, test_svc_no_precosseing = testing(SVC, best_svc,outliers_lof, RandomUnderSampler(), doPCA=False,outliers=False, balancing=False) 
svc_cfs.append((cf_matrix_svc_5,"Test SVC with no preprocessing"))
total_results = total_results.append(pd.Series(['Support Vector Machines',"Test SVC with no preprocessing",
                                test_svc_no_precosseing[0],test_svc_no_precosseing[1],
                                test_svc_no_precosseing[2],test_svc_no_precosseing[3]],index=total_results.columns),ignore_index=True)

              precision    recall  f1-score   support

           0       0.86      0.78      0.82      3079
           1       0.65      0.76      0.70      1676

    accuracy                           0.77      4755
   macro avg       0.76      0.77      0.76      4755
weighted avg       0.79      0.77      0.78      4755

              precision    recall  f1-score   support

           0       0.87      0.77      0.82      3079
           1       0.66      0.79      0.72      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.78      0.77      4755
weighted avg       0.80      0.78      0.78      4755

              precision    recall  f1-score   support

           0       0.86      0.82      0.84      3079
           1       0.69      0.76      0.72      1676

    accuracy                           0.80      4755
   macro avg       0.78      0.79      0.78      4755
weighted avg       0.80      0.80      0.80      4755

              preci

In [ ]:
print_cfs(svc_cfs)

In [ ]:
print_lc(SVC(C=1,kernel='rbf'))

In this case, the Recall metric improved and as a matter of fact the number of misclassifications of class 1 (Hadrons) are less then the previous algorithm. Moreover, the learning curves here increase with the increasing of the training samples, but the test accuracy surpasses the training's, indicating and underfitting model and hence suffering of high bias.

### **K-Nearest Neighbours**
KNN is one of the simplest supervised algorithms in machine learning, and it is based on the assumptions that near points are similar. Specifically, given a positive $K$, it identifies the $K$ points that are closest to the test observation indicated with $N_0$, estimate the conditional probability for the class:
<br/><br/>
\begin{equation}
Pr⁡(Y=j | X=x_0)= \frac{1}{K}  \sum_{i ∈ N_0}I(y_i=j)
\end{equation}
<br/>
Finally, the Bayes rule is applied, and the algorithm classifies the test observation to the class with the highest probability. There are several metrics to evaluate the distance between the test example and the training data points, two can be the Euclidean distance or the Mahalanobis distance:
<br/><br/>
\begin{equation} Euclidean_{dist} = \sqrt{\sum_{i=1}^{m}(x_i-y_i)^2}\end{equation}
<br/>
\begin{equation} Mahalanobis_{dist} = \sqrt{(x- y)^{T}\Sigma^{-1}(x-y)} \end{equation}
<br/>
The choice of $K$ has a key role in the behaviour of the algorithm. As $K$ grows, KNN becomes less flexible, and the decision boundaries tends to be linear, and can easily lead to underfitting because the algorithm is less able to generalize. On the other hand, if $K$ is too small it can result in overfitting because the algorithm is too sensible to possible outliers. 


In [ ]:
params_knn = {"n_neighbors": [15,25,50,100],
              "metric": ["euclidean", "manhattan", "chebyshev"]}

pd_knn, knn = training(KNN, params_knn,outliers_iso, SMOTE())

Done!


In [ ]:
df_knn = print_conf_acc(pd_knn)

In [ ]:
best_knn = chose_best_params(pd_knn)

The best configuration is Configuration 7: {'metric': 'manhattan', 'n_neighbors': 50}


In [ ]:
knn_cfs = []
cf_matrix_knn_1, test_knn_iso_pca_smote = testing(KNN, best_knn,outliers_iso, SMOTE()) 
knn_cfs.append((cf_matrix_knn_1,"Test KNN with Iso, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['K-Nearest Neighbors',"Test KNN with Iso, PCA, SMOTE",
                                test_knn_iso_pca_smote[0],test_knn_iso_pca_smote[1],
                                test_knn_iso_pca_smote[2],test_knn_iso_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_knn_2, test_knn_iso_pca_rus = testing(KNN, best_knn,outliers_iso, RandomUnderSampler()) 
knn_cfs.append((cf_matrix_knn_2,"Test KNN with Iso, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['K-Nearest Neighbors',"Test KNN with Iso, PCA, Random Undersampling",
                                test_knn_iso_pca_rus[0],test_knn_iso_pca_rus[1],
                                test_knn_iso_pca_rus[2],test_knn_iso_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_knn_3, test_knn_lof_pca_smote = testing(KNN,best_knn,outliers_lof, SMOTE()) 
knn_cfs.append((cf_matrix_knn_3,"Test KNN with LOF, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['K-Nearest Neighbors',"Test KNN with LOF, PCA, SMOTE",
                                test_knn_lof_pca_smote[0],test_knn_lof_pca_smote[1],
                                test_knn_lof_pca_smote[2],test_knn_lof_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_knn_4, test_knn_lof_pca_rus = testing(KNN, best_knn,outliers_lof, RandomUnderSampler()) 
knn_cfs.append((cf_matrix_knn_4,"Test KNN with LOF, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['K-Nearest Neighbors',"Test KNN with LOF, PCA, Random Undersampling",
                                test_knn_lof_pca_rus[0],test_knn_lof_pca_rus[1],
                                test_knn_lof_pca_rus[2],test_knn_lof_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_knn_5, test_knn_no_precosseing = testing(KNN, best_knn,outliers_lof, RandomUnderSampler(), doPCA=False,outliers=False, balancing=False) 
knn_cfs.append((cf_matrix_knn_5,"Test KNN with no preprocessing"))
total_results = total_results.append(pd.Series(['K-Nearest Neighbors',"Test KNN with no preprocessing",
                                test_knn_no_precosseing[0],test_knn_no_precosseing[1],
                                test_knn_no_precosseing[2],test_knn_no_precosseing[3]],index=total_results.columns),ignore_index=True)

              precision    recall  f1-score   support

           0       0.85      0.79      0.82      3079
           1       0.66      0.74      0.70      1676

    accuracy                           0.77      4755
   macro avg       0.75      0.77      0.76      4755
weighted avg       0.78      0.77      0.78      4755

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      3079
           1       0.69      0.72      0.70      1676

    accuracy                           0.79      4755
   macro avg       0.77      0.77      0.77      4755
weighted avg       0.79      0.79      0.79      4755

              precision    recall  f1-score   support

           0       0.84      0.83      0.84      3079
           1       0.70      0.71      0.70      1676

    accuracy                           0.79      4755
   macro avg       0.77      0.77      0.77      4755
weighted avg       0.79      0.79      0.79      4755

              preci

In [ ]:
print_cfs(knn_cfs)

In [ ]:
print_lc(KNN(n_neighbors=50,metric='manhattan'))

For KNN algorithm the metrics in the cross validation steps varies similar to the RandomForest's. In fact also the number of misclassification in the test step is higher then SVM. <br/> On the other hand, looking at the learning curves that the model performed well and it is learning from the data until reaching a convergence point.

### **Logistic Regression**
Logistic Regression is a General Linear Model that learns a function $h$ in $R^d$ to the interval $[0,1]$. In fact, this model is used for classification task since the output can be interpreted as a probability distribution. Specifically, $h(x)$ is the probability that $x=1$. There are many functions able to return this can of outputs, in logistic regression the function used is the logistic function:
<br/><br/>
\begin{equation}
p(X)=\frac{e^{β_0+ β_1 X}}{1+e^{β_0+ β_1X}} = \frac{e^{<w,x>}}{1+e^{<w,x>}}
\end{equation}
<br/>
This is called a *sigmoid function* since it will always produce the characteristic “S-shape”. Recall, that $<w,x>$ is a compact format to represent $β_0+ β_1 X$. <br/>
Notice that, every time $<w,x>$ is very large than the logistic function is close to one, while if $<w,x>$ is very small then the logistic function is close to zero. Thus, since the prediction corresponds to the $sign<w,x>$, in the first case the sample is assign to class 1 and in the second case to class 0. After some computation from the above formula we arrive at the following:
<br/><br/>
\begin{equation}
\log⁡{\frac{p(x)}{1-p(X)}}=β_0+β_1 X
\end{equation}<br/>
This quantity is called *log-odds* or *logits*, and it is linear in $X$, which means that increasing the $X$ with one unit the logistic regression function will increase with a quantity $β_1$. The two coefficients $β_0$ and $β_1$ are unknown, to estimated them it is used the method of maximum likelihood, a well-known statistical method to estimate parameters. In this way, the two parameters are chosen to maximize the likelihood function. The Maximum Likelihood corresponds exactly to the optimization of the logistic loss function: 
<br/><br/>
\begin{equation}
argmin_{w∈ R^d}(\frac{1}{m}\sum_{i=1}^{m}log⁡{(1+e^{-y_i<w,x_i>})}
\end{equation}
<br/>
The above have the advantage of being a convex function, allowing in this way to solve the optimization problem with standard methods like gradient descent. 
 


In [ ]:
param_lr={"C": [0.0001,0.001,0.01,0.1,1,10],"solver":["newton-cg", "lbfgs", "liblinear", "sag"]}
pd_lr, lr = training(LogisticRegression, param_lr,outliers_iso, SMOTE())

Done!


In [ ]:
df_lr = print_conf_acc(pd_lr)

In [ ]:
best_lr = chose_best_params(pd_lr)

The best configuration is Configuration 18: {'C': 1, 'solver': 'lbfgs'}


In [ ]:
lr_cfs = []
cf_matrix_lr_1, test_lr_iso_pca_smote = testing(LogisticRegression, best_lr,outliers_iso, SMOTE()) 
lr_cfs.append((cf_matrix_lr_1,"Test LR with Iso, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Logistic Regression',"Test LR with Iso, PCA, SMOTE",
                                test_lr_iso_pca_smote[0],test_lr_iso_pca_smote[1],
                                test_lr_iso_pca_smote[2],test_lr_iso_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_lr_2, test_lr_iso_pca_rus = testing(LogisticRegression, best_lr,outliers_iso, RandomUnderSampler()) 
lr_cfs.append((cf_matrix_lr_2,"Test LR with Iso, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Logistic Regression',"Test LR with Iso, PCA, Random Undersampling",
                                test_lr_iso_pca_rus[0],test_lr_iso_pca_rus[1],
                                test_lr_iso_pca_rus[2],test_lr_iso_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_lr_3, test_lr_lof_pca_smote = testing(LogisticRegression,best_lr,outliers_lof, SMOTE()) 
lr_cfs.append((cf_matrix_lr_3,"Test LR with LOF, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Logistic Regression',"Test LR with LOF, PCA, SMOTE",
                                test_lr_lof_pca_smote[0],test_lr_lof_pca_smote[1],
                                test_lr_lof_pca_smote[2],test_lr_lof_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_lr_4, test_lr_lof_pca_rus = testing(LogisticRegression, best_lr,outliers_lof, RandomUnderSampler()) 
lr_cfs.append((cf_matrix_lr_4,"Test LR with LOF, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Logistic Regression',"Test LR with LOF, PCA, Random Undersampling",
                                test_lr_lof_pca_rus[0],test_lr_lof_pca_rus[1],
                                test_lr_lof_pca_rus[2],test_lr_lof_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_lr_5, test_lr_no_precosseing = testing(LogisticRegression, best_lr,outliers_lof, RandomUnderSampler(), doPCA=False,outliers=False, balancing=False) 
lr_cfs.append((cf_matrix_lr_5,"Test LR with no preprocessing"))
total_results = total_results.append(pd.Series(['Logistic Regression',"Test LR with no preprocessing",
                                test_lr_no_precosseing[0],test_lr_no_precosseing[1],
                                test_lr_no_precosseing[2],test_lr_no_precosseing[3]],index=total_results.columns),ignore_index=True)

              precision    recall  f1-score   support

           0       0.85      0.80      0.82      3079
           1       0.66      0.74      0.70      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.77      0.76      4755
weighted avg       0.78      0.78      0.78      4755

              precision    recall  f1-score   support

           0       0.85      0.80      0.82      3079
           1       0.66      0.74      0.70      1676

    accuracy                           0.77      4755
   macro avg       0.75      0.77      0.76      4755
weighted avg       0.78      0.77      0.78      4755

              precision    recall  f1-score   support

           0       0.84      0.81      0.83      3079
           1       0.68      0.72      0.70      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.77      0.76      4755
weighted avg       0.78      0.78      0.78      4755

              preci

In [ ]:
print_cfs(lr_cfs)

In [ ]:
print_lc(LogisticRegression(C=1,solver='saga'))

In this case, by looking at the learning curves the test trend stands most of the time above the training trend indicating an underfitting model hence which suffers of high bias.

### **QDA**

Quadratic Discriminant Analysis is a classification method that assumes a quadratic decision boundary. The classifier assumes that the observations are taken from a Gaussian distribution, and that each class has its own covariance matrix. So that, each observation from the $k^{th}$ class is of the form $X \sim N(\mu_k, \Sigma_k)$ where $\Sigma_k$ is the covariance matrix for the $ k^{th}$ class. Under this assumption, the Bayes classifier assigns the observation $X=x$ the class for which the quantity $\delta_k(x)$ is the largest. 
<br/><br/>
\begin{equation}
\delta_k(x) = - \frac{1}{2} x^{T}\Sigma_k^{-1}x + \Sigma_k^{-1}\mu_k - \frac{1}{2}\mu_k^T\Sigma_k^{-1}\mu_k -\frac{1}{2}\log{|\Sigma_k|} + \log{\pi_k}
\end{equation}
<br/>
Where $\pi_k$ are the priors of each class and $\mu_k$ are their mean values.
The classifiers gets its name since $x$ appears as a *quadratic* function. 
<br/> QDA is usually reccommended when dealing with large datasets, so that the assumption of a different covariance matrix for each class is resounable. In this way also the variance of the classifiers should not be a major concern.


In [ ]:
 params_qda = {'reg_param':[0.0,0.01,1,0.001]}

pd_qda, qda = training(QDA, params_qda, outliers_iso,SMOTE())

Done!


In [ ]:
df_qda = print_conf_acc(pd_qda)

In [ ]:
best_qda = chose_best_params(pd_qda)

The best configuration is Configuration 2: {'reg_param': 0.01}


In [ ]:
qda_cfs = []
cf_matrix_qda_1, test_qda_iso_pca_smote = testing(QDA, best_qda,outliers_iso, SMOTE()) 
qda_cfs.append((cf_matrix_qda_1,"Test QDA with Iso,PCA,SMOTE"))
total_results = total_results.append(pd.Series(['Quadratic Discriminant Analysis',"Test QDA with Iso,PCA,SMOTE",
                                test_qda_iso_pca_smote[0],test_qda_iso_pca_smote[1],
                                test_qda_iso_pca_smote[2],test_qda_iso_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_qda_2, test_qda_iso_pca_rus = testing(QDA, best_qda,outliers_iso, RandomUnderSampler()) 
qda_cfs.append((cf_matrix_qda_2,"Test QDA with Iso, PCA, Rand UnderSampling"))
total_results = total_results.append(pd.Series(['Quadratic Discriminant Analysis',"Test QDA with Iso, PCA, Rand UnderSampling",
                                test_qda_iso_pca_rus[0],test_qda_iso_pca_rus[1],
                                test_qda_iso_pca_rus[2],test_qda_iso_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_qda_3, test_qda_lof_pca_smote = testing(QDA,best_qda,outliers_lof, SMOTE()) 
qda_cfs.append((cf_matrix_qda_3,"Test QDA with LOF, PCA, SMOTE"))
total_results = total_results.append(pd.Series(['Quadratic Discriminant Analysis',"Test QDA with LOF, PCA, SMOTE",
                                test_qda_lof_pca_smote[0],test_qda_lof_pca_smote[1],
                                test_qda_lof_pca_smote[2],test_qda_lof_pca_smote[3]],index=total_results.columns),ignore_index=True)
cf_matrix_qda_4, test_qda_lof_pca_rus = testing(QDA, best_qda,outliers_lof, RandomUnderSampler()) 
qda_cfs.append((cf_matrix_qda_4,"Test QDA with LOF, PCA, Random Undersampling"))
total_results = total_results.append(pd.Series(['Quadratic Discriminant Analysis',"Test QDA with LOF, PCA, Random Undersampling",
                                test_qda_lof_pca_rus[0],test_qda_lof_pca_rus[1],
                                test_qda_lof_pca_rus[2],test_qda_lof_pca_rus[3]],index=total_results.columns),ignore_index=True)
cf_matrix_qda_5, test_qda_no_precosseing = testing(QDA, best_qda,outliers_lof, RandomUnderSampler(), doPCA=False,outliers=False, balancing=False) 
qda_cfs.append((cf_matrix_qda_5,"Test QDA with no preprocessing"))
total_results = total_results.append(pd.Series(['Quadratic Discriminant Analysis',"Test QDA with no preprocessing",
                                test_qda_no_precosseing[0],test_qda_no_precosseing[1],
                                test_qda_no_precosseing[2],test_qda_no_precosseing[3]],index=total_results.columns),ignore_index=True)

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      3079
           1       0.68      0.70      0.69      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.76      0.76      4755
weighted avg       0.78      0.78      0.78      4755

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      3079
           1       0.68      0.70      0.69      1676

    accuracy                           0.78      4755
   macro avg       0.76      0.76      0.76      4755
weighted avg       0.78      0.78      0.78      4755

              precision    recall  f1-score   support

           0       0.79      0.90      0.84      3079
           1       0.76      0.57      0.65      1676

    accuracy                           0.79      4755
   macro avg       0.78      0.74      0.75      4755
weighted avg       0.78      0.79      0.78      4755

              preci

In [ ]:
print_cfs(qda_cfs)

In [ ]:
print_lc(QDA(reg_param=0.01))

In this case we se yet another situation. The algorithm is not learning anything from the data, on the constrast the performances are slightly descreasing with the increasing of the training examples falling in a high bias model/underfitting

# **Conclusion**

The scope of this project was to analyze how machine learning algorithms works and their performances. We have noticed that the PCA did not improved the performance of every model and the balancing methods actully worsened the performance, possibly due to the fact that the imbalance in the dataset was not high. Another important "problem" due to the imbalance in the dataset is that all algorithms suffer of lower scores in precision f-1 with respect to the accuracy since they predict better the samples beloging to the class gamma ray.

As we can see the SVM algorithm does the best job considering all the different metrics. Moreover, it is clear that the preprocessing steps help to reduce the number of misclassifications, but cause a little drop in the accuracies values.

Below we summerize the results with all the algorithms on the test set: 


In [ ]:
final_results = total_results.copy()
final_results.set_index(['Algorithm', 'Configuration'],drop=True,inplace=True)

In [ ]:
final_results

In [ ]:
#@title

from IPython.core.display import HTML

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: #52b69a' if v else '' for v in is_max]


def hover(hover_color="#fec89a"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
    


styles = [
    hover(),
    dict(selector="th", props=[("text-align", "center"),
                               ("color", '#ffffff'),
                               ("background-color", "#2d6a4f"),
                               ("height", "60px"),
                               ("width", "max-width"),
                               ("font", "Nunito Sans")]),
    dict(selector="tr", props=[("text-align", "center"),
                ("background-color", "#e2ece9"),
                 ("font-color", '#fffff'),
                 ("padding", "20px")])
]

final_results.style.apply(highlight_max).set_properties(**{'width': '200px'}).set_table_styles(styles)